In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# 매번 설치필요
!pip install pytorch_pretrained_bert

     |████████████████████████████████| 133kB 3.9MB/s 
     |████████████████████████████████| 133kB 17.8MB/s 
     |████████████████████████████████| 71kB 6.2MB/s 
     |████████████████████████████████| 7.0MB 15.2MB/s 
ERROR: botocore 1.19.28 has requirement urllib3<1.27,>=1.25.4; python_version != "3.4", but you'll have urllib3 1.24.3 which is incompatible.


In [ ]:
# 작업디렉토리 설정
# 파일을 저장한 공간에 맞게 경로를 수정
# run_classifier_morp.py 가 있는 위치
import os 
os.chdir('/content/drive/MyDrive/Colab Notebooks/kobert_wp/src_examples')

In [ ]:
# pytorch_wordpiece 모델
# etri_bert를 사용한 fine tuning
# 사전학습된 모델을 통해 분류모델을 학습
# vocab.korean.rawtext.list를 통해서 토큰화의 기준이 되는 단어 확인 가능
# data_dir에는 train, dev(test), label 파일이 있어야함
# do_train, do_eval은 입력값 없이 존재만하면 True 값을 리턴(사용하지 않을시 삭제필요)
# bert_model_path에 지정된 폴더에는 model.bin, config 파일이 있어야함
# output_dir의 경우 폴더에 파일이 존재할 시 에러 발생, 파일을 비움 or 새로운 경로지정
!python run_classifier_morp.py \
  --task_name cola \
  --do_train \
  --do_eval \
  --data_dir ./fire \
  --vocab_file vocab.korean.rawtext.list \
  --bert_model_path ./dew \
  --max_seq_length 128 \
  --train_batch_size 4 \
  --learning_rate 2e-5 \
  --num_train_epochs 2.0 \
  --output_dir ./output3

In [ ]:
# f1_score 계산
# list 형식으로 데이터를 입력
# 정답과, 예측 라벨 input
def f1_score(test, predict):

    if len(test) != len(predict):
        return print('예측과 정답의 길이가 다름')

    f1_df = [[0 for i in range(len(set(test)))] for i in range(len(set(test)))]

        
    for i in range(len(test)):
        colums = predict[i]
        idx =  test[i]
        f1_df[idx][colums] += 1

    recall = 0
    for i in range(len(set(test))):
        if sum(f1_df[i]) != 0 :
            recall +=f1_df[i][i]/sum(f1_df[i])
        else:
            recall += 0

    precision = 0
    for i in range(len(set(test))):
        
        total = 0
        for j in range(len(set(test))):
            total += f1_df[j][i]

        if total != 0 :
            precision +=f1_df[i][i]/total
            
        else:
            precision += 0
 
    recall = recall/6
    precision = precision/6
    f1_score = 2*(precision*recall)/(precision+recall)
    return f1_score

In [ ]:
import pandas as pd

# dev에서 정답 라벨을 추출
test  = pd.read_csv('./fire/dev.tsv', sep='\t',encoding = 'utf-8',header =None)
test = [ test[1].values[i] for i in range(len(test))]

# 예측결과가 저장된 output 폴더에서 예측 라벨을 호출
f = open('./output7/eval_results_labels.txt', 'r',encoding = 'utf-8')
predict_list = [int(i) for i in f]
f.close()

f1_score(test, predict_list) # f1계산

0.7252500946784544

In [ ]:
# 단일 문장을 예측하는 코드

import argparse
import csv
import logging
import os
import random
import sys

import numpy as np
import torch
from torch.utils.data import (DataLoader, RandomSampler, SequentialSampler,
                              TensorDataset)
from torch.utils.data.distributed import DistributedSampler
from tqdm import tqdm, trange

from pytorch_pretrained_bert.file_utils import PYTORCH_PRETRAINED_BERT_CACHE
from pytorch_pretrained_bert.modeling import BertForSequenceClassification, BertConfig, WEIGHTS_NAME, CONFIG_NAME 

from src_tokenizer.tokenization import BertTokenizer # 수정
from pytorch_pretrained_bert.optimization import BertAdam, WarmupLinearSchedule # 수정

import urllib3
import json

# 위의 bert 입력값과 동일
data_dir = './fire'
task_name = 'cola'
output_dir = './output1'  
vocab_file = 'vocab.korean.rawtext.list'
cache_dir = ""
max_seq_length = 128
eval_batch_size = 8
local_rank = -1
seed = 42
gradient_accumulation_steps = 1

# 데이터 라벨에 따라 list 값을 수정해야함
label_list = ['0', '1', '2', '3', '4', '5']

tokenizer = BertTokenizer.from_pretrained(vocab_file, do_lower_case=False)

cache_dir = cache_dir if cache_dir else os.path.join(str(PYTORCH_PRETRAINED_BERT_CACHE), 'distributed_{}'.format(local_rank))
model = BertForSequenceClassification.from_pretrained(output_dir,
              cache_dir=cache_dir,
              num_labels = 6) # num_labels_task 라벨의 길이에 맞춰야함 or len(label_list)
model.to(device)
param_optimizer = list(model.named_parameters())
no_decay = ['bias', 'LayerNorm.bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
        {'params': [p for n, p in param_optimizer if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
        {'params': [p for n, p in param_optimizer if any(nd in n for nd in no_decay)], 'wei
         

global_step = 0
nb_tr_steps = 0
tr_loss = 0

############## tokens_a에 예측하고 싶은 문장을 추가

tokens_a = ''
tokens_a = '목격자 김계동(남/42년생)에 의하면 화재 장소 주변에 볼일이 있어서 이동중 공동묘지 부근에서 연기가 발생하고 있어 확인해보니 공동묘지에서 화재가 발생하고 있었다고 진술함.  - 선착대 안면센터, 안면읍의용소방대 현장도착하여, 공동묘지 임야 화재진압 및 연소확대 방지조치 실시함.  - 현장조사한 바, 인적이 드문 공동묘지에서 발생한 화재로 현장에서 실화자를 확인할 수는 없었으나, 현장에서 특별한 발화원이 식별되지 않는 점, 공동묘지에서 발생한 화재인 점으로 보아 미상의 성묘객이 성묘후 발생한 담뱃불이 지피물에 착화되어 발생한 화재로 추정됨. '
tokens_a = tokenizer.tokenize(tokens_a)     # 토큰화

if len(tokens_a) > max_seq_length -2:
    tokens_a = tokens_a[:(max_seq_length-2)] # max_seq_length에 맞게 길이를 조절
tokens = ["[CLS]"] + tokens_a + ["[SEP]"] 
segment_ids = [0] * len(tokens) # 문장이 2개 이어진게 아니기 때문에 문장구분을 안해도됨
input_ids = tokenizer.convert_tokens_to_ids(tokens) # 인덱싱
input_mask = [1] * len(input_ids)
padding = [0] * (max_seq_length - len(input_ids))
input_ids += padding
input_mask += padding
segment_ids += padding
label_id = 3
# input data의 길이가 지정된 길이와 동일하게 고정
assert len(input_ids) == max_seq_length
assert len(input_mask) == max_seq_length
assert len(segment_ids) == max_seq_length


# 전처리
# input하고 싶은 데이터를 전처리해주는 함수를 봐야함 convert_examples_to_features 분해
# tensor 입력
all_input_ids = torch.tensor([input_ids ], dtype=torch.long)
all_input_mask = torch.tensor([input_mask ], dtype=torch.long)
all_segment_ids = torch.tensor([segment_ids], dtype=torch.long)
all_label_ids = torch.tensor([label_id ], dtype=torch.long)

# Dataset 구성 문장인덱스, 마스킹, 문장분류, 라벨
eval_data = TensorDataset(all_input_ids, all_input_mask, all_segment_ids, all_label_ids)
eval_sampler = SequentialSampler(eval_data)
eval_dataloader = DataLoader(eval_data, sampler=eval_sampler, batch_size=eval_batch_size)

# 평가모드
model.eval()

for input_ids, input_mask, segment_ids , label_ids in tqdm(eval_dataloader, desc="Evaluating"):
        input_ids = input_ids.to(device)
        input_mask = input_mask.to(device)
        segment_ids = segment_ids.to(device) # 예측만하면 되서 정답은 생략
        
with torch.no_grad():
        
        logits = model(input_ids, segment_ids, input_mask)

logits = logits.detach().cpu().numpy()
current_out = np.argmax(logits, axis=1)

# 결과값 pre
# dictionary를 통해서 문장으로 바꾸면됨
for key in current_out:
    pre = int(key)
    print(pre)

Evaluating: 100%|██████████| 1/1 [00:00<00:00, 486.47it/s]


2
